In [1]:
import numpy as np
import pandas as pd
import datetime as datetime

# 1. Read the datasets

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
df_train = pd.read_csv("/content/drive/MyDrive/DBL Process Mining/Data/Split/bpi2017_train.csv", parse_dates = ['time:timestamp'])
df_val = pd.read_csv("/content/drive/MyDrive/DBL Process Mining/Data/Split/bpi2017_val.csv", parse_dates = ['time:timestamp'])
df_test = pd.read_csv("/content/drive/MyDrive/DBL Process Mining/Data/Split/bpi2017_test.csv", parse_dates = ['time:timestamp'])
df_train

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621126,621126,Obtained,User_43,W_Call after offers,Workflow,Workitem_162257588,start,2016-11-12 09:31:06.232000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621127,621127,statechange,User_43,A_Complete,Application,ApplState_1435859997,complete,2016-11-12 09:31:06.234000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621128,621128,statechange,User_43,A_Cancelled,Application,ApplState_860885554,complete,2016-11-12 09:35:55.142000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
621129,621129,statechange,User_43,O_Cancelled,Offer,OfferState_2011904028,complete,2016-11-12 09:35:55.161000+00:00,Not speficied,New credit,Application_1583440896,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119


## Assign a postion number to each row/process

In [61]:
def assign_position(df: pd.DataFrame) -> pd.DataFrame:
    # Count number of processes per trace/ID
    count_lst = df.groupby('case:concept:name').count()['lifecycle:transition'].tolist()
    position_lst_1 = [list(range(1, i + 1)) for i in count_lst]
    position_lst = []
    for i in position_lst_1:
        for j in i:
            position_lst.append(j)
    df['position'] = position_lst
    return df

In [62]:
df_train = assign_position(df_train)
df_val = assign_position(df_val)
df_test = assign_position(df_test)

# 2. Baseline Case Prediction (Only on Training Dataset)

## Select the most common action per position

In [77]:
position_df = df_train.groupby('position').agg(pd.Series.mode)['concept:name'].to_frame()

In [105]:
# for positions where mode has > 1 position name, save only the 1st one to the column 'one'
position_df['one'] = position_df['concept:name'].copy()for i in range(1, len(position_df)):
  if type(position_df['concept:name'].loc[i])!= str:
    position_df['one'].loc[i] = position_df['concept:name'].loc[i][0]

## Merge the training dataset with with the predicted actions based on their position

In [107]:
df_train = pd.merge(df_train, position_df, on='position')
df_train

,Unnamed: 0,Action,org:resource,concept:name_x,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,position,concept:name_y,one
0,0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
1,22,Obtained,User_119,W_Validate application,Workflow,Workitem_736976847,start,2016-01-07 09:40:08.560000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
2,47,Created,User_19,O_Create Offer,Offer,Offer_997411923,complete,2016-01-02 11:25:44.202000+00:00,Home improvement,New credit,Application_428409768,15000.0,15000.0,120.0,True,158.98,True,1059.0,15000.0,NaN,1,W_Validate application,W_Validate application
3,65,Released,User_34,W_Call incomplete files,Workflow,Workitem_1289398961,suspend,2016-01-13 14:34:27.526000+00:00,Home improvement,New credit,Application_428409768,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
4,105,Created,User_1,W_Handle leads,Workflow,Workitem_124706175,schedule,2016-01-01 13:22:19.977000+00:00,Existing loan takeover,New credit,Application_1266995739,7000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621126,87223,Obtained,User_24,W_Handle leads,Workflow,Workitem_434387367,start,2016-02-16 13:33:20.797000+00:00,Car,New credit,Application_561387569,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,176,W_Handle leads,W_Handle leads
621127,87224,Obtained,User_24,W_Handle leads,Workflow,Workitem_261625899,start,2016-02-16 13:33:24.635000+00:00,Car,New credit,Application_561387569,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177,W_Handle leads,W_Handle leads
621128,87225,Released,User_24,W_Handle leads,Workflow,Workitem_684049303,suspend,2016-02-16 13:33:44.964000+00:00,Car,New credit,Application_561387569,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,W_Handle leads,W_Handle leads
621129,87226,Obtained,User_17,W_Handle leads,Workflow,Workitem_986702656,resume,2016-02-16 13:38:20.529000+00:00,Car,New credit,Application_561387569,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,179,W_Handle leads,W_Handle leads


In [109]:
# calculate what proportion of predictions is correct
# considering that we are only looking at 'one' - the first position in mode - the accuracy should be viewed
# here as an orientational indication, not precise measure
accuracy_train = len(df_train[df_train['concept:name_x'] == df_train['one']])/len(df_train)
accuracy_train

0.1793116105942225

# 3. Apply pre-computed prediction to Validation and Test datasets

## Validation dataset

In [114]:
# Count number of processes per trace/ID
df_val = pd.merge(df_val, position_df, on='position')

df_val.sort_values(by=['time:timestamp'], inplace=True)
df_val.rename(columns = {"concept:name_y":"baseline_action_pred", "concept:name_x":"concept:name"}, inplace=True)
df_val

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,position,baseline_action_pred,one
0,0,Created,User_1,A_Create Application,Application,Application_1746793196,complete,2016-01-01 12:34:53.911000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
4078,1,statechange,User_1,A_Submitted,Application,ApplState_282472804,complete,2016-01-01 12:34:53.950000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,W_Validate application,W_Validate application
8156,2,Created,User_1,W_Handle leads,Workflow,Workitem_1676837235,schedule,2016-01-01 12:34:54.320000+00:00,Car,New credit,Application_1746793196,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,W_Validate application,W_Validate application
109913,84,Created,User_1,A_Create Application,Application,Application_1017492916,complete,2016-01-01 19:18:43.997000+00:00,Car,New credit,Application_1017492916,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,W_Validate application,W_Validate application
112456,85,statechange,User_1,A_Submitted,Application,ApplState_1185546975,complete,2016-01-01 19:18:44.039000+00:00,Car,New credit,Application_1017492916,15000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,W_Validate application,W_Validate application
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65055,149271,statechange,User_1,O_Cancelled,Offer,OfferState_1327087001,complete,2016-11-13 07:00:37.009000+00:00,Home improvement,Limit raise,Application_1546890088,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_424449331,16,W_Validate application,W_Validate application
69111,149272,Deleted,User_1,W_Call after offers,Workflow,Workitem_1332831233,ate_abort,2016-11-13 07:00:37.018000+00:00,Home improvement,Limit raise,Application_1546890088,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,W_Validate application,W_Validate application
3925,149201,statechange,User_1,A_Cancelled,Application,ApplState_338161915,complete,2016-11-13 07:00:49.818000+00:00,Home improvement,New credit,Application_1792327676,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
8003,149202,statechange,User_1,O_Cancelled,Offer,OfferState_341732870,complete,2016-11-13 07:00:49.840000+00:00,Home improvement,New credit,Application_1792327676,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_230690584,2,W_Validate application,W_Validate application


In [115]:
# calculate what proportion of predictions is correct
accuracy_val = len(df_val[df_val['concept:name'] == df_val['one']])/len(df_val)
accuracy_val

0.17842465532480437

## Test dataset

In [116]:
df_test = pd.merge(df_test, position_df, on="position")
df_test.sort_values(by=['time:timestamp'], inplace=True)
df_test.rename(columns={"concept:name_y":"baseline_action_pred", "concept:name_x":"concept:name"}, inplace=True)
df_test

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,position,baseline_action_pred,one
0,0,Created,User_1,A_Create Application,Application,Application_837911105,complete,2016-11-14 05:57:57.461000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,W_Validate application,W_Validate application
751,1,statechange,User_1,A_Submitted,Application,ApplState_1692094008,complete,2016-11-14 05:57:59.458000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,W_Validate application,W_Validate application
1502,2,Created,User_1,W_Handle leads,Workflow,Workitem_1100633268,schedule,2016-11-14 05:57:59.981000+00:00,Existing loan takeover,New credit,Application_837911105,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,W_Validate application,W_Validate application
17931,25,Created,User_41,A_Create Application,Application,Application_912681536,complete,2016-11-14 08:12:30.523000+00:00,Home improvement,New credit,Application_912681536,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,W_Validate application,W_Validate application
18470,26,Created,User_41,W_Complete application,Workflow,Workitem_1334591020,schedule,2016-11-14 08:12:30.536000+00:00,Home improvement,New credit,Application_912681536,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,W_Validate application,W_Validate application
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24967,3727,Deleted,User_131,W_Call incomplete files,Workflow,Workitem_1308647312,ate_abort,2017-02-01 14:00:30.259000+00:00,Existing loan takeover,New credit,Application_637536789,17250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,W_Validate application,W_Validate application
25203,3728,Created,User_131,W_Validate application,Workflow,Workitem_1339594215,schedule,2017-02-01 14:00:30.272000+00:00,Existing loan takeover,New credit,Application_637536789,17250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,W_Validate application,W_Validate application
25431,3729,Obtained,User_131,W_Validate application,Workflow,Workitem_1697785565,start,2017-02-01 14:00:30.275000+00:00,Existing loan takeover,New credit,Application_637536789,17250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,W_Validate application,W_Validate application
25646,3730,statechange,User_131,A_Validating,Application,ApplState_1576288280,complete,2017-02-01 14:00:30.347000+00:00,Existing loan takeover,New credit,Application_637536789,17250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47,W_Validate application,W_Validate application


In [117]:
# calculate what proportion of predictions is correct
accuracy_test = len(df_test[df_test['concept:name'] == df_test['one']])/len(df_test)
accuracy_test

0.15508952832160525

### For easy exploration/correctness verification:

In [ ]:
# Just dropping columns for easier inspection, this shouldn't go to the final version
# pred_df.drop(pred_df.columns.difference(['Action', 'concept:name_x', 'baseline_action_pred', 'position', 'lifecycle:transition']), 1, inplace=True)
# Save to csv for lookup
# pred_df.to_csv('baseline_prediction.csv')